## This program is written for Screening process of job application at goGame Company by Ramtin Ranji

### I hereby declare that I am the sole writer of this code, and it's written based on the requirements of the screening project. This code has been tested on different machines to ensure its correctness 

#### The code is written by Anaconda, Python 2.7. In order to run the code, the following libraries are required: pandas, sqlite3, lorem and glob

Import requirements, for text generation I use lorem 0.1.1
to install run "pip install lorem" 

In [1]:
#required for reading bunch of files from system
import glob
#required for generating a random text
from lorem.text import TextLorem
#required to read and write json objects
import json
#main python tools for Data Analysis
import pandas as pd
from pandas import Series,DataFrame
#Required for SQL database operation
import sqlite3
#just some other libraries to work with!
from datetime import datetime
import random
import os

Some requirements before actually start the project

In [2]:
## normalize a list of words. All words must be same case and also
## '.' character must be removed from a word to count the unique words
## accurately. Otherwise 'Also' and 'also' might be considered as different 
## words. Same is true about '.Also' and 'Also'
## The only punctuation in loren Ipsum is '.', for the actual text analysis
## project other characters such as ',.!?' must be considered as well
def normalize_text(text_list):
    text_list = text_list.lower()
    temp_text = text_list.replace('.','')
    return temp_text

# A function to generate a random date between 2018-1-1 to 2018-1-31
def random_date():
    day = random.randint(1,31)
    sec = random.randint(0,59)
    minute = random.randint(0,59)
    hour = random.randint(0,23)
    return datetime(2018,1,day,hour,minute,sec)

#Remove previously generated text files, if the code runs more that once
previous_text_files = glob.glob('ramtin*.txt')
for f in previous_text_files:
    os.remove(f)

### 1) Data generation step, change the number of N to your desired value, default set to 1001

In [3]:
#Step 1: Create files
###############################
###############################
###############################

#Initial Value for generating data    
N = 1001

#create N/3 number of users
user_list = []
for i in range(0,N/3):
    temp = '0000000' + str(i)
    user_list.append(temp[-4:])

#A counter to control the filenames, each file must be seprated for each
# 1000 lines of data
file_count = 0

#Generate N number of data
for temp_counter in range(0,N):
    # generate a random date between 2018-1-1 to 2018-1-31
    date = random_date()
    
   # Text size must be random min 10 max 50, so the range of sentence is (5-10) words
   # and the range of paragraph is (2-5) sentences, therefore in the minimum condition
   # we have a paragraph 5*2 = 10 words and in the maximum condition we should have
   # a 10*5 = 50 words 
    lorem = TextLorem(srange =(5,10),prange=(2,5))
    # generate the random Text
    text = lorem.paragraph()
    mmlist = text.split()

    #Build a JSON object, in Python, JSON equals to a dictionary
    record = {"text":text,
              "user_id":user_list[random.randint(0, len(user_list) -1)],
              "datetime":str(date)
              }
    
    #To split file for each 1000 lines of data we divide the current counter by 1000, 
    #and create a new file for each 1000 line of text
    if(temp_counter % 1000 == 0):
        file_count += 1
        
    #Generate the filename, a new filename for each 1000 lines of data
    filename='ramtin'+str(file_count)+'.txt'
    
    #Append to the file
    f = open(filename,'a')
    
    #Write the text as a JSON object to the file and close it
    json.dump(record,f)
    f.write('\n')
    f.close()

### 2) Reading from the files

In [4]:
#Step 2: Read from files
###############################
###############################
###############################

#Our main dataframe to store all fetched data from filesystem
dframe = DataFrame(columns=['text','user_id','datetime'])

for sourceFile in (sorted(glob.glob('ramtin*.txt'))):
    with open(sourceFile) as f:
        for line in f:
            #Read data as JSON object from file
            data = json.loads(line)
            #add JSON object to the main dataframe
            dd = DataFrame(data,index =['0'])
            dframe = pd.concat([dd,dframe],ignore_index = True,sort=False)

### 3) Processing Data

In [5]:
#Step 3: Processing Data
###############################
###############################
###############################   
            
#Normalizing the text to remove '.' character and change all words to lower case
dframe['text'] = map(lambda txt:normalize_text(txt),dframe['text'])

#Add a new column to include the count of each text field in Data Frame
dframe['count'] = map(lambda txt:len(txt),dframe['text'])

#Fix the type of datetime field, Originally it's unicode, here it's changed
#to datetime64
dframe['datetime'] = map(lambda dd:pd.to_datetime(dd),dframe['datetime'])


### 4) Data Analysis and producing reports

In [6]:
#Step 4: Generate Reports
###############################
###############################
###############################

#### Top 10 most frequently used words in all files. Report A

In [7]:
#Report A: Top 10 most frequently used words in all files

#Copy all words into a list
allwords = []
for temp in dframe['text']:
    allwords.extend(temp.split())

#Convert the list to Series and count unique values with their frequency
# in the series which contains all of the words from the files
word_counts = Series(allwords).value_counts()

#It contains 10 most frequently used words
#Note: It might possible that the frequency of the 11th word be the same
# as the 10th word, for example both repeated 100 times. Since you have
# asked for 10 words, I use this method
top10_words =  word_counts.head(10)

#### Showing Sample Result

In [8]:
print top10_words

eius          1017
voluptatem    1011
adipisci      1002
sed           1000
numquam       1000
ipsum          993
etincidunt     992
dolore         992
porro          990
non            986
dtype: int64


#### Most frequently used word grouped by date. Report B

In [9]:
#Report B: Most frequently used word grouped by date.  
    
#Empty dataframe
word_date_frame = DataFrame(columns=['Date','Frequency','Word'])

#We use normalize to omit the effect of hours;minute;Second on the groupby
for a in dframe.groupby(pd.DatetimeIndex(dframe['datetime']).normalize()):
    
    #Store all the words in each day in a list
    all_words_date = []
    for tt in a[1]['text']:
        all_words_date.extend(tt.split())
    
    #Count Unique words with their frequency in each day
    word_series = Series(all_words_date).value_counts()
    
    
    for x in range (word_series.count()):
        #Check to see if the current words is repeated most
        #It might possible that more than one word reated most
        #so it must be checked first
        if word_series[x] == word_series.max():
            #Add a record to the dataframe
            record = {"Date":[a[0]],
              "Frequency":[word_series[x]],
              "Word":[word_series.index.values[x]] 
              }
            temp_frame = DataFrame(record)
            word_date_frame = pd.concat([temp_frame,word_date_frame],ignore_index = True,sort=False)
#Sort values of dataframe by Date
word_date_frame = word_date_frame.sort_values(by='Date')

#### Showing Sample results

In [10]:
print word_date_frame.head(15)

         Date Frequency        Word
33 2018-01-01        42         non
32 2018-01-02        52       velit
31 2018-01-03        33  etincidunt
30 2018-01-04        39    quisquam
29 2018-01-04        39      labore
28 2018-01-04        39      magnam
27 2018-01-05        51  voluptatem
26 2018-01-06        34       dolor
25 2018-01-07        51       neque
24 2018-01-08        35     dolorem
23 2018-01-09        35      dolore
22 2018-01-10        37        eius
21 2018-01-11        51    adipisci
20 2018-01-12        44      labore
19 2018-01-13        34  voluptatem


#### Each User's total word counts. Report C

In [11]:
# Report C: Each User's total word counts
#Since in data processing step, I've created "Count" column
# it can be calculated easily with a groupby on the users and then sum it
user_sum = dframe.groupby(['user_id']).sum()
#create a dataframe to easily pass into the SQL
user_result = DataFrame({'User_id': user_sum.index,'TotalWord':user_sum['count']})

#### Show sample results 

In [12]:
print user_result['TotalWord'].head()

user_id
0001    680
0002    121
0003    472
0004    698
0005    267
Name: TotalWord, dtype: int64


### 5) Write the results into a database

In [13]:
#Step 5: Write the results into a database

#Create a Database if it's not already exist or connect to it if it's existed
sqlconnection = sqlite3.connect('goGame-Ramtin.db')


sql_create_TopTen_table = """ CREATE TABLE IF NOT EXISTS TopTen (
                                        id integer PRIMARY KEY,
                                        Word text NOT NULL,
                                        Frequency integer NOT NULL,
                                        AnalysisDate datetime NOT NULL
                                    ); """

sql_create_MostWords_byDate_table = """ CREATE TABLE IF NOT EXISTS MostWords_byDate (
                                        id integer PRIMARY KEY,
                                        Word text NOT NULL,
                                        Date datetime NOT NULL,
                                        Frequency integer NOT NULL,
                                        AnalysisDate datetime NOT NULL
                                    ); """
sql_create_User_Total_Words_table = """ CREATE TABLE IF NOT EXISTS User_Total_Words (
                                        id integer PRIMARY KEY,
                                        User_id text NOT NULL,
                                        AnalysisDate datetime NOT NULL,
                                        TotalWord integer NOT NULL                                        
                                    ); """
curser = sqlconnection.cursor()

curser.execute(sql_create_TopTen_table)
curser.execute(sql_create_MostWords_byDate_table)
curser.execute(sql_create_User_Total_Words_table)

# record the time of analysis
now = datetime.now()

#Write the results of the Report A:Top 10 most frequently used words in all files
#into the database
report2_frame = DataFrame({'Word':top10_words.index,'Frequency':top10_words.values,'AnalysisDate':now})
report2_frame.to_sql('TopTen',sqlconnection,index=False,if_exists='append')

#Write the results of the Report B:Most frequently used word grouped by date
#into the database
word_date_frame['AnalysisDate'] = now
word_date_frame.to_sql('MostWords_byDate',sqlconnection,index=False,if_exists='append')

#Write the results of the report C: Each User's total word counts
#into the database
user_result['AnalysisDate'] = now
user_result.to_sql('User_Total_Words',sqlconnection,index=False,if_exists='append')

### Read from SQL Database

In [14]:
#SQL query to read data from a table
sql_read_TopTen_query = """ SELECT * FROM TopTen """

#Read table data into a dataframe
df = pd.read_sql(sql_read_TopTen_query,sqlconnection)

#show the first 15 rows of dataframe
print df.head(15)

    id        Word  Frequency                AnalysisDate
0    1     tempora       1014  2019-01-12 23:48:11.380000
1    2     quaerat        989  2019-01-12 23:48:11.380000
2    3       porro        987  2019-01-12 23:48:11.380000
3    4     dolorem        987  2019-01-12 23:48:11.380000
4    5        modi        981  2019-01-12 23:48:11.380000
5    6  etincidunt        979  2019-01-12 23:48:11.380000
6    7    adipisci        978  2019-01-12 23:48:11.380000
7    8       neque        977  2019-01-12 23:48:11.380000
8    9      dolore        973  2019-01-12 23:48:11.380000
9   10         sed        973  2019-01-12 23:48:11.380000
10  11     tempora       1014  2019-01-12 23:56:49.374000
11  12     quaerat        989  2019-01-12 23:56:49.374000
12  13       porro        987  2019-01-12 23:56:49.374000
13  14     dolorem        987  2019-01-12 23:56:49.374000
14  15        modi        981  2019-01-12 23:56:49.374000


## ***** Please contact me if you need any clarification about the code
***** ramtinranji@gmail.com
***** ramtin_ranji@yahoo.com